In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# from webdriver_manager.chrome import ChromeDriverManager
import time
from collections import defaultdict

# 創建一個 Options 物件
chrome_options = Options()

chrome_options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
chrome_options.add_argument("--incognito") # 無痕模式
chrome_options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。
    
service = Service('../4-Selenium/chromedriver.exe')  # chromedriver 路徑

# 使用 ChromeDriverManager 自動管理 chromedriver
# service = Service(ChromeDriverManager().install()) 

# 建立 WebDriver 物件
driver = webdriver.Chrome(service=service, options=chrome_options)

# 進入目標網站
driver.get('https://www.twse.com.tw/zh/trading/historical/stock-day-avg.html')

stock_price_dict = defaultdict(dict)
stock_code_list = ['00934', '00935', '00936']

for stock_code in stock_code_list:
    stock_price_dict[stock_code] = {
        "date": [],
        "price": []
    }
    stock_input = driver.find_element(By.CLASS_NAME, "stock-code-autocomplete")
    stock_input.send_keys(stock_code)

    date_element = driver.find_element(By.NAME, "date")
    driver.execute_script("arguments[0].value='20251001';", date_element)

    # 定位按鈕並點擊
    button_element = driver.find_element(By.CSS_SELECTOR, ".panel.label5 button.search")
    button_element.click()
    
    time.sleep(2)

    price_elements_list = driver.find_elements(By.CSS_SELECTOR, "tbody.is-last-page > tr")

    for element in price_elements_list:
        date_and_price = element.find_elements(By.TAG_NAME, 'td')

        date = date_and_price[0].text
        price = date_and_price[1].text
        if date == '月平均收盤價':
            continue
        
        stock_price_dict[stock_code]["date"].append(date)
        stock_price_dict[stock_code]["price"].append(price)

    
    stock_input.clear()

In [ ]:
print(stock_price_dict)

### 補充 - 資料呈現 (bokeh)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from datetime import datetime
from bokeh.palettes import Category10
import random
from bokeh.models import HoverTool

output_notebook()  # 若在 Jupyter Notebook 使用

p = figure(
    x_axis_type="datetime",
    width=600,
    height=400,
    title="股票價格折線圖",
)

renderers = []

for key, value in stock_price_dict.items():
    # 將民國日期轉換為西元日期
    stock_price_dict[key]["date"] = [str(int(date[:3]) + 1911) + date[3:] for date in stock_price_dict[key]["date"]]
    stock_price_dict[key]["date"] = [datetime.strptime(date, "%Y/%m/%d") for date in stock_price_dict[key]["date"]]
    
    color = random.choice(Category10[10])  # 10 個漂亮顏色隨便挑
    
    # 繪製折線圖
    p.line(stock_price_dict[key]["date"], stock_price_dict[key]["price"], line_width=2, color=color, legend_label=f"{key}")
    # 繪製散點圖，方便滑鼠提示
    circle = p.scatter(stock_price_dict[key]["date"], stock_price_dict[key]["price"], marker="circle", size=6, color=color, name=key)
    # 將散點圖加入 renderers，供 HoverTool 使用
    renderers.append(circle)

# 加入滑鼠提示工具（HoverTool）
hover = HoverTool(
    tooltips=[
        ("股票", "$name"),
        ("日期", "@x{%F}"),
        ("價格", "@y{0,0.00}"),
    ],
    formatters={
        "@x": "datetime",  # x 軸是 datetime
    },
    mode="vline",  # 垂直線模式：對齊同一日期的所有點
    renderers=renderers,
)
p.add_tools(hover)

p.xaxis.axis_label = "日期"
p.yaxis.axis_label = "價格"

# 顯示圖表
show(p)

### 補充 - 使用 API 抓取資料

In [ ]:
import requests

url = "https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY_AVG?date=20250630&stockNo=0050&response=json"

response = requests.get(url)
data = response.json()

print(data)